In [1]:
import ast
import mmcv
import math
import torch
import torch.nn.functional as F
import torchvision
import pickle as pkl
import pandas as pd
import numpy as np
import seaborn as sns
from einops import rearrange
import matplotlib.pyplot as plt

from torchmetrics.functional.classification import (
    multilabel_f1_score,
    multilabel_precision,
    multilabel_recall,
)

from data_utils import results2df
from sklearn.metrics.pairwise import cosine_similarity

# Slowfast imports
from slowfast.models import build_model
from slowfast.utils.parser import load_config, alt_parse_args
from slowfast.datasets.loader import construct_loader

/home/dl18206/anaconda3/envs/slowfast/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/dl18206/anaconda3/envs/slowfast/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
path_to_config = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/configs/SLOW_8x8_R50_Local.yaml"
path_to_ckpt = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/checkpoint_epoch_00200.pyth"

args = alt_parse_args()[:-1]
cfg = load_config(
    args[0],
    path_to_config=path_to_config,
)
checkpoint = torch.load(path_to_ckpt, map_location="cpu")

model = build_model(cfg)
model.load_state_dict(checkpoint["model_state"])
model.eval()
model.cpu()

ResNet(
  (s1): VideoModelStem(
    (pathway0_stem): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
  )
  (s2): ResStage(
    (pathway0_res0): ResBlock(
      (branch1): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (branch1_bn): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (branch2): BottleneckTransform(
        (a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (a_bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (a_relu): ReLU(inplace=True)
        (b): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), 

In [3]:
classifier = model.head.projection

In [4]:
loader = construct_loader(cfg, "test")  # dataset = build_dataset("tap", cfg, "train")
sample = next(iter(loader))

SAMPLING.BALANCED: False; BALANCE_TYPE: None


/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/slowfast/datasets/decoder.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  video_tensor = torch.from_numpy(np.frombuffer(video_handle, dtype=np.uint8))
/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/slowfast/datasets/decoder.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for t

In [5]:
inputs, labels, index, time, meta = sample

In [8]:
inputs[0].shape

torch.Size([1, 3, 16, 256, 256])

In [ ]:
meta, labels

In [ ]:
feature_map = model.s5(
    model.s4(model.s3(model.s2(model.s1([inputs[0][1].unsqueeze(0)]))))
)[0]

In [ ]:
def extract_frame_wise_features(feature_map, t):
    spatially_pooled = F.adaptive_avg_pool3d(feature_map, (t, 1, 1))
    frame_wise_features = torch.flatten(spatially_pooled, start_dim=2)
    return frame_wise_features


frame_wise_features = extract_frame_wise_features(feature_map, t=16)

In [ ]:
# Compute frame-wise cosine similarity
frame_wise_features = frame_wise_features.squeeze(0)
frame_wise_features = frame_wise_features.T
frame_wise_features = F.normalize(frame_wise_features, p=2, dim=1)
frame_wise_cosine_similarity = torch.mm(frame_wise_features, frame_wise_features.T)

In [ ]:
video_level_features = F.adaptive_avg_pool3d(feature_map, (1, 1, 1))
video_level_features = torch.flatten(video_level_features, start_dim=1)

In [ ]:
# Plot cosine similarity matrix
# plt.figure(figsize=(10, 10))
# sns.heatmap(frame_wise_cosine_similarity.detach().numpy(), cmap="viridis", annot=True)
# plt.title("Frame-wise cosine similarity")
# plt.show()

In [ ]:
frame_wise_logits = []
for feat in frame_wise_features:
    frame_wise_logits.append(torch.sigmoid(classifier(feat).detach()).numpy())

In [ ]:
# Plot frame-wise logits as heatmap
frame_wise_logits = np.array(frame_wise_logits)
# plt.figure(figsize=(10, 15), dpi=300)
# sns.heatmap(frame_wise_logits.T, cmap="viridis", annot=True)
# plt.title("Frame-wise logits")
# plt.show()

In [ ]:
video_level_logits = torch.sigmoid(classifier(video_level_features)).detach().numpy()

video_level_logits = np.array(video_level_logits)
# plt.figure(figsize=(1, 10))
# sns.heatmap(video_level_logits.T, cmap="viridis", annot=True)
# plt.title("Video-level logits")
# plt.show()

In [ ]:
import cv2

In [ ]:
# Get original video
video_path = "/home/dl18206/Desktop/phd/data/panaf/panaf_sequence/36070505.mp4"
cap = cv2.VideoCapture(video_path)
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

In [ ]:
# Reshape frame[10] to 256x256
frame = frames[10]
frame = cv2.resize(frame, (256, 256))
frame.shape

In [ ]:
# View frame
plt.imshow(frame)
plt.axis("off")

In [ ]:
# Spatial


def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h * w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [ ]:
spatial_map = feature_map[:, :, 0, :, :]

In [ ]:
cams = returnCAM(
    spatial_map.detach(),
    classifier.weight.detach().numpy(),
    torch.linspace(0, 13, steps=14).int(),
)

In [ ]:
# render the CAM and output
height, width, _ = frame.shape
heatmap = cv2.applyColorMap(cv2.resize(cams[10], (width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + frame * 0.5
cv2.imwrite("CAM.jpg", result)